# Turkish ULMFiT from scratch

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [3]:
bs=128
torch.cuda.set_device(2)
data_path = Config.data_path()

lang = 'tr'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

## Turkish wikipedia model

In [4]:
from nlputils import split_wiki,get_wiki

get_wiki(path,lang)
!head -n4 {path}/{name}

/home/jhoward/.fastai/data/trwiki/trwiki already exists; not downloading
<doc id="10" url="https://tr.wikipedia.org/wiki?curid=10" title="Cengiz Han">
Cengiz Han

Cengiz Han ("Cenghis Khan", "Çinggis Haan" ya da doğum adıyla Temuçin (anlamı: demirci), Moğolca: "Чингис Хаан" ya da "Tengiz" (anlamı: deniz), ; d. 1162 – ö. 18 Ağustos 1227), Moğol komutan, hükümdar ve Moğol İmparatorluğu'nun kurucusudur. Cengiz Han, 13. Yüzyılın başında Orta Asya'daki tüm göçebe bozkır kavimlerini birleştirerek bir ulus haline getirdi ve o ulusu "Moğol" siyasi kimliği çatısı altında topladı. Dünya tarihinin en büyük askeri dehalarından biri olarak kabul edilen Cengiz Han, hükümdarlığı döneminde 1206-1227 arasında Kuzey Çin'deki Batı Xia ve Jin Hanedanı, Türkistan'daki Kara Hıtay, Maveraünnehir, Harezm, Horasan ve İran'daki Harzemşahlar ile Kafkasya'da Gürcüler, Deşt-i Kıpçak'taki Rus Knezlikleri ve Kıpçaklar ile İdil Bulgarları üzerine gerçekleştirilen seferler sonucunda Pasifik Okyanusu'ndan Hazar Denizi’

In [ ]:
dest = split_wiki(path,lang)

/home/jhoward/.fastai/data/trwiki/docs already exists; not splitting


Turkish is an [Agglutinative_language](https://en.wikipedia.org/wiki/Agglutinative_language) so it needs special care!

![Turkish morphemes example](images/turkish.jpg)

In [ ]:
%%time
data = (TextList.from_folder(dest, processor=[OpenFileProcessor(), SPProcessor(n_cpus=0)])
        .split_by_rand_pct(0.1, seed=42)
        .label_for_lm()
        .databunch(bs=bs, num_workers=1))

In [ ]:
data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

In [ ]:
data.show_batch()

In [ ]:
# data = load_data(path, f'{lang}_databunch', bs=bs)

In [12]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.1, wd=0.1, pretrained=False).to_fp16()

In [10]:
lr = 3e-3
lr *= bs/48  # Scale learning rate by batch size

In [11]:
learn.unfreeze()
learn.fit_one_cycle(3, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,19.272875,19.598858,0.000006,10:26


KeyboardInterrupt: 

Save the pretrained model and vocab:

In [ ]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

## Turkish sentiment analysis

https://www.win.tue.nl/~mpechen/projects/smm/

In [35]:
train_df = pd.read_csv(path/'train.csv')
train_df.loc[pd.isna(train_df.comment),'comment']='NA'
train_df.head()

,id,comment,label
0,train_000000,Dung dc sp tot cam on \nshop Đóng gói sản phẩm...,0
1,train_000001,Chất lượng sản phẩm tuyệt vời . Son mịn nhưng...,0
2,train_000002,Chất lượng sản phẩm tuyệt vời nhưng k có hộp ...,0
3,train_000003,:(( Mình hơi thất vọng 1 chút vì mình đã kỳ vọ...,1
4,train_000004,Lần trước mình mua áo gió màu hồng rất ok mà đ...,1


In [36]:
test_df = pd.read_csv(path/'test.csv')
test_df.loc[pd.isna(test_df.comment),'comment']='NA'
test_df.head()

,id,comment
0,test_000000,Chưa dùng thử nên chưa biết
1,test_000001,Không đáng tiềnVì ngay đợt sale nên mới mua n...
2,test_000002,Cám ơn shop. Đóng gói sản phẩm rất đẹp và chắc...
3,test_000003,Vải đẹp.phom oki luôn.quá ưng
4,test_000004,Chuẩn hàng đóng gói đẹp


In [37]:
df = pd.concat([train_df,test_df], sort=False)

In [38]:
data_lm = (TextList.from_df(df, path, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [17]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [39]:
lr = 1e-3
lr *= bs/48

In [19]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.975080,4.138585,0.317773,00:07
1,4.408635,4.025489,0.326423,00:07


In [20]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.142114,3.928278,0.336230,00:09
1,4.010835,3.793583,0.349972,00:09
2,3.873617,3.694702,0.357240,00:09
3,3.761377,3.632186,0.364648,00:09
4,3.679017,3.595601,0.366964,00:09
5,3.614548,3.576386,0.369224,00:09
6,3.575895,3.567496,0.370285,00:09
7,3.560278,3.566525,0.370173,00:10


In [21]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [20]:
data_clas = (TextList.from_df(df, path, vocab=data_lm.vocab, cols='text')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='pos')
    .databunch(bs=bs, num_workers=1))

data_clas.save(f'{lang}_textlist_class')

In [10]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)

In [ ]:
from sklearn.metrics import f1_score

@np_func
def f1p(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))

@np_func
def f1n(inp,targ): return f1_score(1-targ, np.argmin(inp, axis=-1))

In [22]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [23]:
lr=2e-2
lr *= bs/48

In [24]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.673509,0.659224,0.578799,00:01
1,0.650918,0.638635,0.631332,00:01


In [25]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.650401,0.647810,0.638837,00:01
1,0.640086,0.637472,0.607880,00:01


In [26]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.631687,0.624111,0.667917,00:02
1,0.587725,0.529850,0.725141,00:01


In [27]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.611662,0.611376,0.661351,00:02
1,0.544891,0.487765,0.760788,00:02


In [28]:
learn_c.unfreeze()
learn_c.fit_one_cycle(5, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.479387,0.439782,0.769231,00:02
1,0.437735,0.424604,0.797373,00:02
2,0.393978,0.382782,0.814259,00:02
3,0.367813,0.370652,0.833959,00:02
4,0.350672,0.370038,0.843340,00:02


In [50]:
learn_c.save(f'{lang}clas')

## fin